![CoSAppLogo](images/cosapp.svg) **CoSApp** examples:

# FMI

The Functional Mock-up Interface (FMI) is a free standard that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file.

For more, please look at the official website: https://fmi-standard.org/

The goal is to allow the co-simulation of model from different tools. This notebook presents how to package a CoSApp model as a Functional Mock-up Unit (FMU).

## System without iterative

### Case description

#### Creating the elements

You need first to get your CoSApp system to be defined in a separated file.

In [ ]:
from exprampode_fmu import ExpRampOde

# Trick to display nicely the class code
from IPython.display import Markdown
import inspect
Markdown(f"```py\n{inspect.getsource(ExpRampOde)}```")

#### Export to FMU

Exporting a CoSApp system is done through the function `cosapp.tools.to_fmu`.

Any internal parameter you set before calling the transformation function, will have
that value as default value in the FMU.

In [ ]:
import os
import tempfile
import shutil

from cosapp.tools import to_fmu

syst = ExpRampOde('fmu1')

# Customize parameters
syst.a = 5

if not os.path.exists('fmus'):
    os.mkdir('fmus')

# Export the FMU in a temporary directory
with tempfile.TemporaryDirectory() as tmp_dir:
    fmu_file = to_fmu(syst, dest=tmp_dir, locals=['f', 'df_dt'])
    shutil.copy2(fmu_file, 'fmus')


#### Use the FMU

In [ ]:
import pyfmi

fmu = pyfmi.load_fmu('fmus/ExpRampOde.fmu')
results = fmu.simulate(final_time=10.)

In [ ]:
# Plot results
import plotly.graph_objs as go

layout = go.Layout(
    height = 450,
    xaxis = dict(title="time"),
    yaxis = dict(title = "f")
)

go.Figure(
    data=go.Scatter(
        x = results['time'], 
        y = results['f'],
    ), 
    layout=layout)